In [1]:
from datetime import date
import pandas as pd
from bokeh.io import output_file, show
from bokeh.models import BasicTicker, ColorBar, ColumnDataSource, LinearColorMapper, PrintfTickFormatter
from bokeh.plotting import figure,save
from bokeh.transform import transform
from bokeh.io import export_png

In [2]:
def prepare_dataset(df):
    #Date
    df["Date"]=pd.to_datetime(df["InvoiceDate"]).apply(lambda x:x.date())
    df["Year-Month"]=df["Date"].apply(lambda x: date(x.year,x.month,1))
    
    #sales calculation
    df["Sales"]=df["Quantity"]*df["UnitPrice"]
    
    return df


def cohort_analysis(df):
    df=df.dropna(subset=['CustomerID'])
    year_month=sorted(list(df['Year-Month'].unique()))
    
    #for year 2011
    df1=df[df["Year-Month"].isin(year_month[10:])]
    
    customer_retention=[]
    avg_sales_customer=[]
    
    for dates in year_month[10:]:
        #customer acquired in that month
        customer_acquire=list(df1[df1["Year-Month"]==dates]["CustomerID"].unique())
        no_of_customer=len(customer_acquire)
        
        #behaviour of those customer (customer retention month wise)
        df2=df1[df1["CustomerID"].isin(customer_acquire)]
        df3=df2[["CustomerID","Date","Year-Month"]].drop_duplicates() # n purchase on same day will be count only once
        df3=df3.sort_values("Date")
        
        diff=[]
        for ids in customer_acquire:
            df4=df3[df3["CustomerID"]==ids].reset_index()
            for date in df4["Date"]:  #Note the zero diff gap between first payment would also be included 
                diff.append(round((date-df4["Date"][0]).days/30))
        
        
        retention={}
       
        for month in diff:
            if month in retention:
                retention[month]=retention[month]+1
            else:
                retention[month]=1
        
        
        
        
        #avg sales 
        df5=df2[["CustomerID","Date","Sales"]]
        df5=df5.sort_values("Date")
            
        avg_sales={}
        
        
    
        for ids in customer_acquire:
            df6=df5[df5["CustomerID"]==ids].reset_index()
            df6["Diff_Date"]=df6["Date"].apply(lambda x: round((x-df6["Date"][0]).days/30))
            diff_dates=sorted(list(df6["Diff_Date"].unique()))
            for diff_date_itr in diff_dates:
                df7=df6[df6["Diff_Date"]==diff_date_itr]
                sum_sales=df7["Sales"].sum()
                if diff_date_itr in avg_sales:
                    avg_sales[diff_date_itr]=avg_sales[diff_date_itr]+sum_sales
                else:
                    avg_sales[diff_date_itr]=sum_sales
        
        #average sales per customer
        for key in retention:
            avg_sales[key]=round(avg_sales[key]/retention[key],2)
            
        
        
        
        
        avg_sales["Date"]=dates
        
        #Customer lifetime value
        avg_sales_customer.append(avg_sales)
    
        
        retention[0]=retention[0]-no_of_customer #remove zero gap with same date    
        for key in retention.keys():
            retention[key]=round((retention[key]/no_of_customer)*100,2)
            
        retention["Date"]=dates
        retention["total"]=no_of_customer
        
        #Customer Retention
        customer_retention.append(retention)
        
        
        #remove those customer which are taken in analysis
        df1=df1[~df1["CustomerID"].isin(customer_acquire)]
        
    
    
    #----------------------------------------------------------------------
    #creating Dataframe for visualization

    df8=pd.DataFrame(customer_retention)
    df9=pd.DataFrame(avg_sales_customer)
    col_name=list(df8.columns)
    col_name=col_name[:12]
    data=[]
    for itr in range(0,len(df8)):
        for name in col_name:
            dicts={}
            dicts['Date']=(df8['Date'].iloc[itr]).strftime('%B %Y') +", No of cutomers in cohort= "+ str(df8['total'].iloc[itr])
            dicts['Month']="month-"+str(name)
            dicts['retention']=df8[name].iloc[itr]
            #for nan values (empty string as text)
            if df9[int(name)].iloc[itr]!=df9[int(name)].iloc[itr]:
                dicts['avg_sales']=""
            else:
                dicts['avg_sales']=str(df9[int(name)].iloc[itr])+"$"
            data.append(dicts)
    
    return pd.DataFrame(data)

def visualization(df):
    output_file("cohort_analysis.html")
    
    
    source = ColumnDataSource(df)
    
    colors = ['#084594', '#2171b5', '#4292c6', '#6baed6', '#9ecae1', '#c6dbef', '#deebf7', '#f7fbff']
    mapper = LinearColorMapper(palette=list(reversed(colors)), low=0, high=100,nan_color='white') #customer retention percentage 0 - 100

    p = figure(plot_width=1200, plot_height=620, title="                                        Cohort Analysis",
               y_range=list(df["Date"].unique()), x_range=list(reversed(df["Month"].unique())),
               toolbar_location=None, tools="", x_axis_location="below", y_axis_location="right")
    
    p.rect(y="Date", x="Month", width=1, height=1, source=source,
           line_color='White', fill_color=transform('retention', mapper))
    
    color_bar = ColorBar(color_mapper=mapper, location=(0, 0),
                         ticker=BasicTicker(desired_num_ticks=len(colors)),
                         formatter=PrintfTickFormatter(format="%d%%"))
    
    
    p.text(y="Date", x="Month", source=source, x_offset=-24, text='avg_sales',text_font_size={'value': '8pt'})
    p.add_layout(color_bar, 'left')
    p.axis.axis_line_color = None
    p.axis.major_tick_line_color = None
    p.axis.major_label_text_font_size ="10pt" 
    p.axis.major_label_standoff = 0
    p.grid.grid_line_color=None
    p.title.text_font_size="15pt"
    show(p)
    


In [3]:
df=pd.read_csv("Online Retail.csv")
dataframe=prepare_dataset(df)
dataframe_analysis=cohort_analysis(dataframe)
visualization(dataframe_analysis)
